In [ ]:
aws_access_key = 'Access_key'
aws_secret_key = 'Secret_key'
region = 'us-east-1'

import boto3

def get_client(service):
    return boto3.client(service,
                        aws_access_key_id=aws_access_key,
                        aws_secret_access_key=aws_secret_key,
                        region_name=region)

In [ ]:
def create_bucket_with_acls(bucket_name):
    s3 = get_client('s3')

    try:
       
        if region == 'us-east-1':
            s3.create_bucket(Bucket=bucket_name)
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"✅ Bucket created: {bucket_name}")

    
        s3.put_bucket_ownership_controls(
            Bucket=bucket_name,
            OwnershipControls={
                'Rules': [{'ObjectOwnership': 'BucketOwnerPreferred'}]
            }
        )
        print(f"✅ Enabled ACLs for: {bucket_name}")

        return True

    except Exception as e:
        print(f"❌ Error creating bucket with ACLs: {e}")
        return False


In [ ]:
def disable_block_public_access(bucket_name):
    s3 = get_client('s3')
    try:
        s3.put_public_access_block(
            Bucket=bucket_name,
            PublicAccessBlockConfiguration={
                'BlockPublicAcls': False,
                'IgnorePublicAcls': False,
                'BlockPublicPolicy': False,
                'RestrictPublicBuckets': False
            }
        )
        print(f"✅ Disabled Block Public Access for: {bucket_name}")
    except Exception as e:
        print(f"❌ Error disabling block settings: {e}")


In [ ]:
def make_bucket_public(bucket_name):
    s3 = get_client('s3')
    try:
        s3.put_bucket_acl(Bucket=bucket_name, ACL='public-read')
        print(f"⚠️ Made bucket public: {bucket_name}")
    except Exception as e:
        print(f"❌ Error setting public ACL on {bucket_name}: {e}")

In [ ]:
test_buckets = ['audit-test-bucket-01', 'audit-test-bucket-02', 'audit-test-bucket-03']

for b in test_buckets:
    create_bucket_with_acls(b)

bucket_name = 'audit-test-bucket-04'

if create_bucket_with_acls(bucket_name):
    disable_block_public_access(bucket_name)   
    make_bucket_public(bucket_name)            

In [ ]:
def get_default_vpc_id():
    ec2 = get_client('ec2')
    vpcs = ec2.describe_vpcs(Filters=[{'Name': 'isDefault', 'Values': ['true']}])
    return vpcs['Vpcs'][0]['VpcId']

default_vpc_id = get_default_vpc_id()

In [ ]:
def create_security_group(group_name, description, vpc_id):
    ec2 = get_client('ec2')
    try:
        response = ec2.create_security_group(
            GroupName=group_name,
            Description=description,
            VpcId=vpc_id
        )
        sg_id = response['GroupId']
        print(f"✅ Created Security Group: {group_name} | ID: {sg_id}")
        return sg_id
    except Exception as e:
        print(f"❌ Error creating security group {group_name}: {e}")

In [ ]:
def add_open_inbound_rule(sg_id, port=80, protocol='tcp'):
    ec2 = get_client('ec2')
    try:
        ec2.authorize_security_group_ingress(
            GroupId=sg_id,
            IpPermissions=[
                {
                    'IpProtocol': protocol,
                    'FromPort': port,
                    'ToPort': port,
                    'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
                }
            ]
        )
        print(f"⚠️ Added open rule to SG {sg_id} for {protocol.upper()} {port}")
    except Exception as e:
        print(f"❌ Error adding rule to {sg_id}: {e}")

In [ ]:
sg_ids = []
sg_ids.append(create_security_group("audit-sg-01", "Test open SG", default_vpc_id))
sg_ids.append(create_security_group("audit-sg-02", "Another test SG", default_vpc_id))


add_open_inbound_rule(sg_ids[0], port=22)  